# Grype DB for Java Vulnerabilities
This notebook search in the <b>Grype database</b> for vulnerabilities that affect all the maven packages stored in the <b>sqlite database</b>. It can be executed only ONCE for all the packages stored.
<hr>

In [ ]:
# In order to obtain further information about vulnerabilities, NVD API are used. The public rate limit (without an API key) 
# is 5 requests in a rolling 30 second window, thus in this notebook the default wait_time is setted to 6 sec between to requests.
# If you want to speed up the process (up to 50 requests in a rolling 30 second window), you can get an API KEY by following the
# instructions in #https://nvd.nist.gov/developers/start-here#:~:text=to%20in%20sequence.-,Request%20an%20API%20Key,-On%20the%20API.

# Once the key is obtained, it is sufficient to paste it in the following variable for increasing the rate limit used in this notebook.
# If you do not want to use a KEY simply leave the following variable EMPTY.
nvd_api_key = "<NVD-API-KEY>" 

#### Requirements
<hr>

#### Logger set up

In [ ]:
import logging, os, datetime,sys
from pathlib import Path
Path('logs').mkdir(parents=True,exist_ok=True)
# Logging Levels: DEBUG, INFO, WARNING, ERROR, CRITICAL
logging.basicConfig(#filename=os.path.join('logs','sbom_creator',str(datetime.datetime.now().strftime("%d-%m-%Y T%H %M %S")) +'.log'),
handlers=[
        logging.FileHandler(os.path.join('logs','log-'+str(datetime.datetime.now().strftime("%d-%m-%Y")) +'.log')),
        logging.StreamHandler(sys.stdout)
    ],
                    format='%(asctime)s |:| LEVEL:%(levelname)-2s |:| FILE:notebook_3.1 (java_grype).ipynb:%(lineno)-s |:| %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logging.getLogger("urllib3").propagate = False

#### Database connection

In [ ]:
from lib.sqlite_utils import DBConnection 

if not os.path.exists(os.path.join('database','database.sqlite')):
    logging.critical('Database does not exists! You need to create it first (db_builder.ipynb)')
    raise Exception('Database does not exists! You need to create it first (db_builder.ipynb)')

conn=DBConnection(os.path.join('database','database.sqlite'))
logging.info('Connected with "database/database.sqlite" database.')

#### Grype database connection

In [ ]:
import os

if not os.path.exists(os.path.join('grype_db','vulnerability.db')):
    logging.critical('Cannot find Grype "vulnerability.db" in "grype_db" folder.')
    raise Exception('Cannot find Grype "vulnerability.db" in "grype_db" folder.')

grype_db_conn=DBConnection(os.path.join('grype_db','vulnerability.db'))
logging.info('Connected with "grype_db/vulnerability.db" database.')

#### Collection of Java vulnerabilities

In [ ]:
#Getting all Java packages and all Grype vulnerabilities:
packages = conn.get_rows('package',dic={'package_manager':'maven'})
grype_vulns = grype_db_conn.get_rows('vulnerability')

In [ ]:
from packageurl import PackageURL
from lib.vuln_utils import is_vulnerable,extend_vulns_with_grypedb
import ast

vulnerabilities = list()
grype_pot_affection = list()

for package in packages:
    purl_dict = PackageURL.from_string(package['purl']).to_dict()
    query_search = (purl_dict['namespace'] + ':' + purl_dict['name']).lower()
    logging.info(f'Querying Grype db for "{query_search}" Java package...')
    vulns = grype_db_conn.query(f'SELECT v.id,v.package_name,m.severity,m.urls,v.version_constraint,v.related_vulnerabilities,v.fixed_in_versions FROM vulnerability v LEFT JOIN vulnerability_metadata m ON v.id=m.id WHERE v.namespace="github:language:java" and LOWER(v.package_name) LIKE "%{query_search}"')
    vulns = [dict(zip(['id','package_name','severity','urls','version_constraint','related_vulnerabilities','fixed_in_versions'],v)) for v in vulns if is_vulnerable(v[4],purl_dict['version'])]
    
    vulns = extend_vulns_with_grypedb([{
        'url':'',
        'other_sources':'',
        'namespace':v['id'].split('-')[0],
        'severity':v['severity'],
        'fixed_package_version':ast.literal_eval(v['fixed_in_versions'])[0] if v['fixed_in_versions']!='None' and v['fixed_in_versions']!= None and len(ast.literal_eval(v['fixed_in_versions']))>0 else '',
        'source':'GRYPE_DB',
        'id': v['id']
    } for v in vulns],grype_vulns)
    vulnerabilities.extend(vulns)
    grype_pot_affection.extend([{
                'vulnerability':v['id'],
                'package':package['purl']
            } for v in vulns])

logging.info('Vulnerabilities obtained for Java packages.')

#### Store vulnerabilities and affections

In [ ]:
logging.info('Storing vulnerabilities and affections in "grype_db/vulnerability.db" database.')
for vuln in vulnerabilities:
    conn.add_or_update('vulnerability',vuln)

for affection in grype_pot_affection:
    conn.add_or_update('grype_potential_affection',affection)

for affection in grype_pot_affection:
    conn.add_or_update('grype_cpe_potential_affection',affection)

#### Use OSV API to store more info about vulnerabilities collected with Grype

In [ ]:
from lib.vuln_utils import extend_vulns_with_nvdapi
logging.info('Getting more info about vulnerabilities with OSV API')
vulnerabilities = extend_vulns_with_nvdapi(vulnerabilities,wait_time=0.6 if nvd_api_key!='' else 6, logger=logging,nvd_api_key=nvd_api_key if nvd_api_key!='' else None)

#### Store vulnerabilities

In [ ]:
logging.info('Storing vulnerabilities in "grype_db/vulnerability.db" database.')
for vuln in vulnerabilities:
    conn.add_or_update('vulnerability',vuln)

#### Close databases

In [ ]:
conn.close()
grype_db_conn.close()